In [1]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [2]:
year_num = 20 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [3]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

In [4]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [5]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [6]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [7]:
yearfile = yearfiles[-1]
print(yearfile)
i=0
tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
print(len(tar.getmembers()[1:]))
#for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
for member in tar.getmembers()[1:]:
    name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
    usaf = name_parts[0]
    wban = int(name_parts[1])
    if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
        i=i+1
        #if i%(stat_num//10) == 0: print(i)
        f=tar.extractfile(member)
        f=gzip.open(f, 'rb')
        content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
        content=preprocess_station_file_content(content)
        df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
        content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
        df = df.append(content)
tar.close()

gsod_2019.tar
11738


In [8]:
df

,USAF,WBAN,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY
0,010030,99999,2019,1,16.00,8.00,12.90,20.80,12.40,16.0
1,010030,99999,2019,2,18.10,9.65,12.75,21.85,13.20,14.5
2,010030,99999,2019,3,9.40,2.70,12.70,14.00,6.10,16.0
3,010030,99999,2019,4,24.65,19.75,8.70,30.75,18.25,8.5
0,719330,99999,2019,1,-2.50,-9.40,5.60,28.20,-11.20,16.0
...,...,...,...,...,...,...,...,...,...,...
3,719077,99999,2019,4,0.30,-3.70,7.70,9.50,-7.60,8.5
0,122350,99999,2019,1,30.50,28.60,7.60,34.00,27.90,16.0
1,122350,99999,2019,2,37.65,33.20,8.00,42.55,30.80,14.5
2,122350,99999,2019,3,40.40,34.10,8.30,45.90,34.50,16.0
